In [2]:
# from openai import OpenAI
import openai
from time import sleep
from functools import cache

EMBED_MODEL = "text-embedding-ada-002"

@cache
def create_embeddings(texts):
    try:
        res = openai.Embedding.create(input=texts, engine=EMBED_MODEL)                    
    except Exception as e:
        print(e)
        done = False
        while not done:
            sleep(5)
            try:
                texts = texts[:]
                res = openai.Embedding.create(input=texts, engine=EMBED_MODEL)
                done = True
            except Exception as e:
                print(e)
                pass
    return res['data'][0]['embedding']

print(create_embeddings('hello world'))

[-0.0149070480838418, 0.0013432145351544023, -0.018515290692448616, -0.031130973249673843, -0.024322718381881714, 0.00748644582927227, -0.022966334596276283, -0.0010074109304696321, -0.012740785256028175, -0.02245275303721428, 0.02575811557471752, 0.010857651010155678, -0.03313262760639191, -0.003703716676682234, 0.005869979504495859, 0.013879884034395218, 0.01939759962260723, -0.02282147854566574, 0.018462616950273514, 0.011265883222222328, -0.006215659435838461, 0.00879015401005745, 0.006620599422603846, -0.00813829991966486, -0.014670009724795818, -0.010963001288473606, 0.01835726574063301, -0.013577002100646496, 0.019265910610556602, -0.039242934435606, 0.0024905437603592873, -0.004717711824923754, -0.017791008576750755, -0.014814866706728935, 0.010344069451093674, -0.009369580075144768, 7.705788448220119e-05, -0.018291421234607697, 0.02416469343006611, -0.019595129415392876, 0.00931032095104456, -0.0012428027112036943, 0.01750129647552967, -0.021794315427541733, -0.038136757910251

In [3]:
import pandas as pd

df = pd.read_csv('NAICS descriptions - Sheet1.csv')
embeds = []

for sector, definition, decscription in zip(df['Sector'], df['Definition'], df['Descriptions']):
    embeds.append({
        'label': sector,
        'definition': definition,
        'embed': create_embeddings(f'{definition} {decscription}')
    })

The server is overloaded or not ready yet.


In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

@cache
def create_embeddings(texts):
    try:
        res = openai.Embedding.create(input=texts, engine=EMBED_MODEL)                    
    except Exception as e:
        print(e)
        done = False
        while not done:
            sleep(5)
            try:
                texts = texts[:len(texts) // 2]
                res = openai.Embedding.create(input=texts, engine=EMBED_MODEL)
                done = True           
            except Exception as e:
                print(len(texts), e)
                pass
    return res['data'][0]['embedding']

def get_similarity(text):
    input_embedding = np.array(create_embeddings(str(text)))
    
    similarities = []
    for item in embeds:
        item_embedding = np.array(item['embed'])
        similarity_score = cosine_similarity(input_embedding.reshape(1, -1), item_embedding.reshape(1, -1))[0][0]
        similarities.append((similarity_score, item['label']))

    similarities.sort(key=lambda x: x[0], reverse=True)

    return [score for score, label in similarities]

# Example usage
similarities = get_similarity("some text")
similarities

[0.7442615204665873,
 0.7412074487121783,
 0.7391098182248445,
 0.7387526865673026,
 0.7385901816641678,
 0.7383574602525416,
 0.7340718814228965,
 0.7324910884331797,
 0.7318998841200288,
 0.731643092309576,
 0.72944579657697,
 0.7274981855935352,
 0.7258685461490851,
 0.7243592138971456,
 0.7236131585967069,
 0.7233438017295861,
 0.7209835840053641,
 0.7189822533567493,
 0.7164385174714084,
 0.7132130228333058]

In [10]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv('combined_df.csv')
new_rows = []

# Process all rows
for i in range(len(df)):
    readme = str(df['readme'][i])
    description = str(df['description'][i])
    our_topic = str(df['topic (search)'][i])
    try:    
        topic = literal_eval(str(df['topics'][i]))
        if our_topic in topic:
            topic.remove(our_topic)
        topic = ' '.join(topic)
    except:
        topic = str(df['topics'][i])
    if not topic:
        topic = 'not sure'
    
    label = df['NAICS Code'][i]
    
    new_row = {
        'readme': readme,
        'description': description,
        'label': label,
        'similarity_readme': get_similarity(readme),
        'similarity_description': get_similarity(description), 
        'similarity_topic': get_similarity(topic), 
        'embedding_readme': create_embeddings(readme),
        'embedding_description': create_embeddings(description),
        'embedding_topic': create_embeddings(topic),
    }
    new_rows.append(new_row)
    
    if i % 10 == 0:
        print(f'Finished {i} rows')

# Create a new DataFrame from the processed rows
combined_df = pd.DataFrame(new_rows)
combined_df.to_csv('combined_df_similarity.csv', index=False)

# Load and display the updated DataFrame
updated_df = pd.read_csv('combined_df_similarity.csv')
updated_df


Finished 0 rows
Finished 10 rows
Finished 20 rows
Finished 30 rows
Finished 40 rows
Finished 50 rows
Finished 60 rows
Finished 70 rows
Finished 80 rows
Finished 90 rows
Finished 100 rows
Finished 110 rows
Finished 120 rows
Finished 130 rows
Finished 140 rows
Finished 150 rows
Finished 160 rows
Finished 170 rows
Finished 180 rows
Finished 190 rows
Finished 200 rows
Finished 210 rows
Finished 220 rows
Finished 230 rows
Finished 240 rows
Finished 250 rows
Finished 260 rows
Finished 270 rows
Finished 280 rows
Finished 290 rows
Finished 300 rows
Finished 310 rows
Finished 320 rows
Finished 330 rows
Finished 340 rows
Finished 350 rows
Finished 360 rows
Finished 370 rows
Finished 380 rows
Finished 390 rows
Finished 400 rows
Finished 410 rows
Finished 420 rows
Finished 430 rows
Finished 440 rows
Finished 450 rows
Finished 460 rows
Finished 470 rows
Finished 480 rows
Finished 490 rows
Finished 500 rows
Finished 510 rows
Finished 520 rows
Finished 530 rows
Finished 540 rows
Finished 550 rows
Fin

,readme,description,label,similarity_readme,similarity_description,similarity_topic,embedding_readme,embedding_description,embedding_topic
0,"<div align=""center"">\n <a href=""https://erp...",Free and Open Source Enterprise Resource Plann...,['44-45'],"[0.7077418639119603, 0.7034992376043662, 0.700...","[0.7552436941405053, 0.7423780541827908, 0.738...","[0.7539479295821286, 0.7521461707911956, 0.750...","[0.005275495816022158, 0.0016473194118589163, ...","[0.013344024308025837, -0.04136915132403374, -...","[0.008875333704054356, -0.0118060652166605, -0..."
1,# Shopizer 3.X (for java 1.8 +) (tested with J...,Shopizer java e-commerce software,['44-45'],"[0.6729698208353336, 0.6725300366484664, 0.666...","[0.7339337416907676, 0.7338221616706524, 0.714...","[0.692403025402172, 0.6911045951984289, 0.6874...","[0.015184788964688778, -0.004053920973092318, ...","[-0.005936639849096537, -0.025300830602645874,...","[-0.003159537445753813, -0.004670771304517984,..."
2,# Forecasting Best Practices \n\nTime series f...,Time Series Forecasting Best Practices & Examples,['44-45'],"[0.6928727293070945, 0.6870969795016502, 0.676...","[0.7342316954393573, 0.7129032797801069, 0.711...","[0.7208858743694974, 0.7023878873172714, 0.699...","[-0.006438827142119408, -0.011148449033498764,...","[-0.006866960786283016, -0.0164752509444952, 0...","[-0.015779273584485054, -0.015530127100646496,..."
3,## Retail Store on Blockchain\n\n### About\n\n...,:moneybag: Retail Store that runs on Ethereum,['44-45'],"[0.7276523206058009, 0.710884471696231, 0.7086...","[0.7343180022256028, 0.7224105752103289, 0.721...","[0.724115340141763, 0.7066868989742153, 0.6947...","[0.004726836457848549, -0.0016325792530551553,...","[0.0011983871227130294, -0.029932500794529915,...","[0.00016907176177483052, -0.004170134663581848..."
4,# OpenPapyrus\n\nwww.petroglif.ru\n\n# En\n\n#...,"Sophisticated ERP, CRM, Point-Of-Sale, etc. Op...","['44-45', '42']","[0.7166755120665116, 0.7134934670109578, 0.711...","[0.7361216209179431, 0.7333972122202823, 0.722...","[0.7593477788104767, 0.7591742967822079, 0.751...","[-0.011811600998044014, -0.012127534486353397,...","[0.007078312803059816, -0.026548802852630615, ...","[-0.0034282291308045387, -0.008458567783236504..."
...,...,...,...,...,...,...,...,...,...
1226,# Twitter-Sentiment-Analysis--Canadian-Electio...,Sentiment Analysis is a branch of Natural Lang...,['81'],"[0.7231339405636654, 0.7084747724129122, 0.707...","[0.7373106395924293, 0.7212078032957091, 0.720...","[0.7243629257216184, 0.7234123749826961, 0.722...","[-0.01950402371585369, 0.005641820374876261, 0...","[-0.022413544356822968, 0.011609099805355072, ...","[-0.0013284225715324283, -0.019282402470707893..."
1227,# Sandstorm Specification\n\nSandstorm is an o...,Specification for an Organizational Tool for P...,['81'],"[0.6943312812808318, 0.6930267034240098, 0.690...","[0.7401196388954598, 0.7344296436430272, 0.731...","[0.7243629257216184, 0.7234123749826961, 0.722...","[-0.03184935450553894, -0.01864900439977646, -...","[-0.03142248094081879, -0.02207990549504757, -...","[-0.0013284225715324283, -0.019282402470707893..."
1228,"{'message': 'Not Found', 'documentation_url': ...",Old Wordpress Theme for a Football Political o...,['81'],"[0.6388411844406965, 0.6383990422500619, 0.633...","[0.713735060459783, 0.7036302193335269, 0.7029...","[0.7243629257216184, 0.7234123749826961, 0.722...","[-0.011292933486402035, -0.008167508989572525,...","[-0.0030056366231292486, -0.007199788000434637...","[-0.0013284225715324283, -0.019282402470707893..."
1229,# Election Database Schema Design\nPerformance...,Performance of some analytics on real data fro...,['81'],"[0.7223308577690909, 0.7143659575087355, 0.714...","[0.7639174429034709, 0.7467780468547823, 0.744...","[0.6951824279025733, 0.6858348394268183, 0.683...","[0.009329571388661861, 0.010630057193338871, -...","[-0.002663189545273781, -0.013983011245727539,...","[-0.016339382156729698, 0.015856146812438965, ..."


In [11]:
import pandas as pd

df = df.dropna()
df = pd.read_csv('combined_df_similarity.csv')

finished_df = df.dropna()
finished_df.to_csv('finished_df.csv', index=False)

In [12]:
finished_df['similarity_description'][0]

'[0.7552436941405053, 0.7423780541827908, 0.7384873853224077, 0.730542613196665, 0.7298939484140295, 0.723064765585362, 0.7229809695601128, 0.7223431124787496, 0.721519268013681, 0.7188896186103988, 0.7178477853901627, 0.7171451259337679, 0.7159354686572683, 0.7143859162369377, 0.7140626361027883, 0.7123509822073588, 0.712262232628224, 0.7102921521473365, 0.709693446108466, 0.702553063246684]'

In [13]:
import ast

convert = pd.read_csv('NAICS descriptions - Sheet1.csv').to_dict()['Sector']
convert = {v:k for k, v in convert.items()}

for index, row in finished_df.iterrows():
    vector = [0] * 20
    for j in ast.literal_eval(row['label']):
        if j == '0': continue 
        vector[convert[j]] = 1

    finished_df.at[index, 'label'] = vector

finished_df.to_csv('openai_finished_df.csv', index=False)